In [1]:
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
import plaidml.keras
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
from keras import Sequential
from keras import regularizers
from keras.layers import LSTM, Dense, LSTM, Flatten, BatchNormalization, Dropout, GRU
from keras.utils import to_categorical
#import tensorflow as tf
import plaidmodule as module
# Load the TensorBoard notebook extension
#%load_ext tensorboard

In [2]:
countriesOfInterest = {"HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ'}
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]
#countriesOfInterest = list(countriesOfInterest & {"ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"})

enc = OneHotEncoder()
enc.fit(np.array(countriesOfInterest).reshape(-1, 1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [3]:
seconds = 5
samplerate = 100
train_x, train_labels, val_x, val_labels, class_weights = module.getSamples(20, 1, seconds, samplerate, countriesOfInterest,
               enc, verbose = 0)

### Fit model

In [4]:
model = keras.Sequential()
model.add(LSTM(32, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True,
#               recurrent_dropout = .5,
          #     kernel_regularizer=regularizers.l2(0.01),
              ))
model.add(BatchNormalization())
model.add(LSTM(32, 
             #  input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = False,
#               recurrent_dropout = .5,
               go_backwards = True
          #     kernel_regularizer=regularizers.l2(0.01),
              ))
model.add(BatchNormalization())
#model.add(Dropout(.5))
model.add(Dense(16, activation= "relu"))
model.add(BatchNormalization())
#model.add(Dropout(.5))
model.add(Dense(len(enc.categories_[0]), activation= "softmax"))
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
print(model.summary())

INFO:plaidml:Opening device "opencl_amd_gfx804.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 500, 32)           7424      
_________________________________________________________________
batch_normalization_1 (Batch (None, 500, 32)           128       
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
batch_normalization_3 (Batch (None, 16)                64        
_________________________________________________________________
dense_2 (Dense)              (None, 12)                204       
Total para

In [ ]:
i = 0

#add recurrent dropout to add noise to duration
desc = "LSTM32_32_Dense16_Output_NoReg_NoDropout_2Step_100train_01LR_NH_5Sec"
log_dir = os.path.join(
    "logs",
    "betterSampling",
    desc
)
model_dir = os.path.join(
    "pickle",
    "save"
)

train_n = 100
val_n  = int(2*train_n/5)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 2
iterations = 10
learn_rate = 0.01

model.fit(train_x, train_labels,
                  epochs = i * epochs + epochs, 
                  initial_epoch = i * epochs,
                  shuffle = True,
                  validation_data = (val_x, val_labels),
                  batch_size = 512,
                  class_weight = class_weights,
         #        callbacks=[tensorboard_callback],
                 verbose = 1)

Train on 12824 samples, validate on 501 samples
Epoch 1/2


INFO:plaidml:Analyzing Ops: 1649 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 2952 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 4345 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 5791 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 7284 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 8994 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 11132 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 13668 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 16820 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 20957 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 22870 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 24325 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 25883 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 27580 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 29389 of 96524 operations complete
INFO:plaidml:Analyzing Ops: 31325 of 96524 operations complet

In [ ]:
#add recurrent dropout to add noise to duration
desc = "LSTM32_32_Dense16_Output_NoReg_NoDropout_2Step_100train_01LR_NH_5Sec"
log_dir = os.path.join(
    "logs",
    "betterSampling",
    desc
)
model_dir = os.path.join(
    "pickle",
    "save"
)

train_n = 100
val_n  = int(2*train_n/5)
epochs = 2
iterations = 10
learn_rate = 0.01
batch_size = 2048
module.train(iterations, learn_rate, train_n, val_n, seconds, samplerate,
                countriesOfInterest, enc, epochs, model_dir,
                model, batch_size)